In [1]:
# importing the libraries
import numpy as np
import cv2 as cv
from sklearn.metrics import pairwise

In [2]:
# defining some global variables

# the background image needs to be recognized to determine 
# if hand has entered the frame
background = None

# parameter taken by accumulate weighted function
accumulated_weight = 0.5

# frame coordinates where hand will be detected
topY = 50
topX = 350
bottomY = 300
bottomX = 600

In [3]:
# working to understand the background and create an average of it
# so that when anything else enters the frame it can detect it
def calAverageBackground(newFrame, accumulated_weight):
    
    global background
    
    if background is None:
        background = newFrame.copy().astype('float')
        return None
    
    cv.accumulateWeighted(newFrame, background, accumulated_weight)

In [4]:
# get the thresholded hand image on which we can apply convex hull 

def segment(roi, threshVal = 35):
    
    # calculate the difference betweeen avg background 
    # and the new roi where our hand must be introduced
    diff = cv.absdiff(background.astype('uint8'), roi)
    
    # apply threshold
    ret, threshed = cv.threshold(diff,
                                 threshVal,
                                 255,
                                 cv.THRESH_BINARY)
    
    # find external contours 
    # here we will get many contours but assuming that
    # area of hand contour is going to be maximum of the found contours
    image, contours, hierarchy = cv.findContours(threshed,
                                                cv.RETR_EXTERNAL,
                                                cv.CHAIN_APPROX_SIMPLE)
    
    
    # if no contours are found
    if len(contours) == 0:
        return None
    else:
        # getting the biggest contour 
        # which we assume is going to be of the hand
        # "Contout area method" calculates the area from the contours list
        handSegement = max(contours, key=cv.contourArea)
        
        # return the thresholded roi and its external contour
        return (threshed, handSegement)

In [5]:
# "Convex hull" draws a polygon by connecting most external points in a contour

def countFingers(thresh, hand):
    
    convHull = cv.convexHull(hand)
    
    # the convex hull contains x and y coordinates of the outermost points
    # we will get the uppermost, lowermost, leftmost and rightmost point
    
    # convHull[:,:,1] => grabs the second elements from all pairs e.g., 2 from [1,2]
    # argmin() returns the index of the least value among these second elements
    smallest_Y_valued_pair_index = convHull[:, :, 1].argmin()
    largest_Y_valued_pair_index = convHull[:, :, 1].argmax()
    smallest_X_valued_pair_index = convHull[:, :, 0].argmin()
    largest_X_valued_pair_index = convHull[:, :, 0].argmax()
    
    # using that index we grab the desired least or most valued coordinate
    topMost = tuple(convHull[smallest_Y_valued_pair_index][0])
    bottomMost = tuple(convHull[largest_Y_valued_pair_index][0])
    leftMost = tuple(convHull[smallest_X_valued_pair_index][0])
    rightMost = tuple(convHull[largest_X_valued_pair_index][0])
    
    
    # index 1 grabs y-coordinate and index 0 grabs x-coordinate
    # considering that the center of palm is
    # halfway between leftMost and rightMost point
    # and 
    # halfway between topMost and bottomMost point
    
    xCenter = (leftMost[0] + rightMost[0]) // 2
    yCenter = (topMost[1] + bottomMost[1]) // 2
    
    # calculating euclidean distances between center and the outermost  points
    # using the pairwise function by sklearn
    # returns a 2D array containing a single 1D array so we index it 
    distances = pairwise.euclidean_distances([[xCenter, yCenter]],
                                             Y=[leftMost, rightMost, topMost, bottomMost])[0]
    # take the max of these distances
    # then use some percentage of it as a radius to draw a circle
    maxDistance = distances.max()
    
    radius = int(0.75 * maxDistance)
    circumference = (2 * np.pi * radius)
    
    # create a blank image
    # then draw a circle with radius as 90% of maxDistance with a thickness of 10
    # then pass the thresholded picture through the circle image (mask)
    circle_roi = np.zeros(thresh.shape, dtype='uint8')
    circle_roi = cv.circle(circle_roi, (xCenter, yCenter), radius, 255, 25)
    circle_roi = cv.bitwise_and(thresh, thresh, mask=circle_roi)
    
    # Trying Erosion
#     kernel = np.ones((3,3), dtype=np.uint8)
#     eroded_img = cv.erode(circle_roi.copy(), kernel, iterations = 1)
    
    # find contours from the circumference of the circle
    # depending on how many fingers we catch in the circumference
    # the count will increase
    image, contours, hierarchy = cv.findContours(circle_roi,
                                             cv.RETR_EXTERNAL,
                                             cv.CHAIN_APPROX_NONE)
    numOfFingers = 0
    
    for cn in contours:
        
        (x,y,w,h) = cv.boundingRect(cn)
        
        # if the contour detected anything in the wrist then ignore
        notTheWrist = (yCenter + 0.2 * yCenter) > (y+h)
        
        # if the contour has too many points
        # which means the object enclosed by the contour points is bigger
        # then amount of points detected should be less than 25% of circumference
        # of the circle_roi (some math there !!)
        limitPoints = ((circumference * 0.25) > cn.shape[0])
        
        
        if notTheWrist and limitPoints:
            numOfFingers += 1
    
    return numOfFingers

In [9]:
cam = cv.VideoCapture(0)

grabbedFrames = 0

while True:
    # grab a frame
    ret, frame = cam.read()
    # create a copy for display purposes
    frameCopy = frame.copy()
    # grab the ROI where hand will be placed
    roi = frame[topY:bottomY, topX:bottomX]
    # convert to grayScale
    gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    # apply blur to reduce noise
    gray = cv.GaussianBlur(gray, (7,7), 0)
    # keep accumulating the background until 60 frames are grabbed
    if grabbedFrames < 120:
        
        calAverageBackground(gray, accumulated_weight)
        
        # keep displaying the warning till accumulation
        if grabbedFrames <= 119:
            
            cv.putText(frameCopy,
                      'Wait ! Accumulating background',
                      (topX, bottomY+10),
                      cv.FONT_HERSHEY_SIMPLEX,
                      1,
                      (0,0,255),
                      2)
    else:
        
        # get the contours and threshed image
        hand = segment(gray)
        # if no contours are found
        if hand is not None:
            # tuple unpacking
            threshed, handSegment = hand
            # draw contours 
            # make sure that contour points are calculated on roi
            # so drawing them on original frame needs addition of
            # the displacement of the roi in the original frame
            cv.drawContours(frameCopy,
                           [handSegment+(topX, topY)],
                           -1,
                           (0,255,0),
                           5)
            
            # get the number of fingers
            count = countFingers(threshed, handSegment)
            
            # write the finger count
            cv.putText(frameCopy,
                      str(count),
                      (topX,topY+5),
                      cv.FONT_HERSHEY_SIMPLEX,
                      1,
                      (0,0,255),
                      2)
            
            # display the threshold image of hand separately
            cv.imshow('Threshold', threshed)
    # draw rectangle on the original frame
    cv.rectangle(frameCopy,
                (topX, topY),
                (bottomX, bottomY),
                (0,255,0),
                5)
    # increment the frame count
    grabbedFrames += 1
    # display the window
    cv.imshow('Finger Counter', frameCopy)
    
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cam.release()
cv.destroyAllWindows()